# Data Collection

In order to begin analyzing stocks we first need to collect data to analyze. We will do this by scraping a specified timeframe of stock information for a specified number of stocks.


## Getting Started
To get started, we must import some libraries:

In [34]:
import sys
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
print("======================")


import yfinance as yf
import pandas as pd

print ("Successfully imported libraries")

Successfully imported libraries


We will begin by specifying a list of tickers we want to scrape:

In [35]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

From here, we have to define some options:

- Interval: This represents the frequency we want to get stock data
- Time Frame (2 options):
    - A)
        - Period: This represents a timeframe in english (1m, 5m, 1d, 1y, ...)
    - B)
        - Start Date: This is the starting date for data scraping
        - End Date: This is the ending date for data scraping

In [36]:
period = '10y'
start_date = None
end_date = None
interval = "1d"

Just a quick check to ensure variables are instantiated correctly:

In [37]:
if tickers == None:
    raise Exception("You must specify a list of tickers to scrape")

if period == None and start_date == None and end_date == None:
            raise Exception("You must specify one timeframe in order to scrape")
        
if period == None and (start_date == None or end_date == None):
    raise Exception("You must specify both ends of the timeframe in order to scrape")

if period not in ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']:
    raise Exception("Please input a valid period")

if interval not in ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']:
    raise Exception("Please input a valid time interval")

if period != None and (start_date != None or end_date != None):
    raise Exception("You can only specify one type of timeframe in order to scrape")

print("Passed!")

Passed!


## Scraping Data
Now that we have defined the scraping parameters, we can actually begin to scrape. We do this by calling the YFinance download method for each stock. However, we have to consider two cases:
- We are using period
- We are using start/end dates


In [38]:
ticker_string = ' '.join(tickers) #convert from list to space-separated string
print(f"Ticker String: \n{ticker_string}")

if period != None: #if using period
    data = yf.download(
        ticker_string,
        period = period,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )
else:
    data = yf.download( #if using start/end dates
        ticker_string,
        start = start_date, 
        end = end_date,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )


data = data.drop([(i, 'Close') for i in tickers], axis=1)
data = data.rename({"Adj Close": "Close"}, axis=1)


data.head()

Ticker String: 
AMZN TSLA BABA HD TM NKE MCD LOW SBUX JD PDD BKNG GM MELI TJX NIO F LULU HMC CMG
[*********************100%***********************]  20 of 20 completed


NIO                             LULU                        \
           Open High Low Close Volume       Open       High        Low   
Date                                                                     
2011-10-26  NaN  NaN NaN   NaN    NaN  52.410000  53.930000  51.150002   
2011-10-27  NaN  NaN NaN   NaN    NaN  55.720001  58.439999  54.919998   
2011-10-28  NaN  NaN NaN   NaN    NaN  57.000000  58.500000  56.560001   
2011-10-31  NaN  NaN NaN   NaN    NaN  57.709999  59.459999  56.410000   
2011-11-01  NaN  NaN NaN   NaN    NaN  54.509998  55.240002  52.570000   

                                ...        BKNG                          \
                Close   Volume  ...        Open        High         Low   
Date                            ...                                       
2011-10-26  53.160000  3319000  ...  510.109985  513.609985  480.320007   
2011-10-27  57.580002  4757300  ...  512.380005  524.419983  510.690002   
2011-10-28  57.939999  2237400  ...  515.000000  526.750000  511.000000   
2011-10-31  56.480000  3207200  ...  519.000000  522.000000  507.690002   
2011-11-01  54.849998  3972900  ...  492.079987  499.230011  478.649994   

                                      SBUX                                   \
                 Close   Volume       Open       High        Low      Close   
Date                                                                          
2011-10-26  495.630005  1489300  20.865000  21.094999  20.360001  17.521368   
2011-10-27  520.549988  1543000  21.235001  21.715000  21.004999  18.228140   
2011-10-28  523.679993   995100  21.485001  21.610001  21.070000  17.986910   
2011-10-31  507.720001   971700  21.135000  21.565001  21.075001  17.927662   
2011-11-01  489.019989  1475900  20.665001  20.934999  20.459999  17.449419   

                      
              Volume  
Date                  
2011-10-26  12060000  
2011-10-27  12129600  
2011-10-28  11935400  
2011-10-31  10543000  
2011-11-01  15438800  

[5 rows x 100 columns]

# Adding Technical Indicators

Using the [TA-Lib](https://mrjbq7.github.io/ta-lib/) package, we can broadcast technical indicators across the time series into their own dataframe columns

In [39]:
import talib
!{sys.executable} -m pip install TA-Lib

def computeRSI(df):
    return talib.RSI(df["Close"], timeperiod=14)

def computeUltimateOscillator(df):
    return talib.ULTOSC(df["High"], df["Low"], df["Close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

def computeBollingerBands(df):
    upperband, middleband, _ = talib.BBANDS(df["Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    return (upperband - middleband) / df["Close"] # need to normalize so value is not dependent on share price

def computeChaikinOscillator(df):
    return talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"], fastperiod=3, slowperiod=10)

def computeNATR(df):
    return talib.NATR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeSMA(df, window):
    return talib.SMA(df["Close"], timeperiod=window) / df["Close"] # need to normalize so value is not dependent on share price

def computeSAR(df):
    return talib.SAR(df["High"], df["Low"], acceleration=0, maximum=0) / df["Close"] # need to normalize so value is not dependent on share price

def computeWilliamsR(df):
    return talib.WILLR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeAPO(df):
    return talib.APO(df["Close"], fastperiod=12, slowperiod=26, matype=0) / df["Close"]# need to normalize so value is not dependent on share price

def computeROC(df, window):
    return talib.ROCP(df["Close"], timeperiod=window) 


Now, we can apply these functions across the collected timeseries

In [40]:
for ticker in tickers:
    df = data[ticker]
    data[ticker, 'RSI'] = computeRSI(df)
    data[ticker, 'Ultimate'] = computeUltimateOscillator(df)
    data[ticker, 'BandRadius'] = computeBollingerBands(df)
    data[ticker, 'Chaikin'] = computeChaikinOscillator(df)
    data[ticker, 'NATR'] = computeNATR(df)
    data[ticker, 'SMA_5'] = computeSMA(df, 5)
    data[ticker, 'SMA_20'] = computeSMA(df, 20)
    data[ticker, 'SMA_100'] = computeSMA(df, 100)
    data[ticker, 'SAR'] = computeSAR(df)
    data[ticker, 'Williams%R'] = computeWilliamsR(df)
    data[ticker, 'APO'] = computeAPO(df)
    data[ticker, 'ROC_5'] = computeROC(df, 5)
    data[ticker, 'ROC_20'] = computeROC(df, 20)
    data[ticker, 'ROC_100'] = computeROC(df, 100)

data.head(100)

NIO                             LULU                        \
           Open High Low Close Volume       Open       High        Low   
Date                                                                     
2011-10-26  NaN  NaN NaN   NaN    NaN  52.410000  53.930000  51.150002   
2011-10-27  NaN  NaN NaN   NaN    NaN  55.720001  58.439999  54.919998   
2011-10-28  NaN  NaN NaN   NaN    NaN  57.000000  58.500000  56.560001   
2011-10-31  NaN  NaN NaN   NaN    NaN  57.709999  59.459999  56.410000   
2011-11-01  NaN  NaN NaN   NaN    NaN  54.509998  55.240002  52.570000   
...         ...  ...  ..   ...    ...        ...        ...        ...   
2012-03-14  NaN  NaN NaN   NaN    NaN  73.339996  73.730003  71.599998   
2012-03-15  NaN  NaN NaN   NaN    NaN  72.580002  74.360001  71.959999   
2012-03-16  NaN  NaN NaN   NaN    NaN  74.360001  74.500000  71.959999   
2012-03-19  NaN  NaN NaN   NaN    NaN  72.320000  73.940002  71.540001   
2012-03-20  NaN  NaN NaN   NaN    NaN  72.610001  73.250000  71.949997   

                                ...       CMG                                \
                Close   Volume  ...      NATR     SMA_5    SMA_20   SMA_100   
Date                            ...                                           
2011-10-26  53.160000  3319000  ...       NaN       NaN       NaN       NaN   
2011-10-27  57.580002  4757300  ...       NaN       NaN       NaN       NaN   
2011-10-28  57.939999  2237400  ...       NaN       NaN       NaN       NaN   
2011-10-31  56.480000  3207200  ...       NaN       NaN       NaN       NaN   
2011-11-01  54.849998  3972900  ...       NaN  1.025090       NaN       NaN   
...               ...      ...  ...       ...       ...       ...       ...   
2012-03-14  72.430000  1748700  ...  1.358906  0.996935  0.976379       NaN   
2012-03-15  74.239998  2160300  ...  1.392901  0.983969  0.963248       NaN   
2012-03-16  72.040001  2796500  ...  1.375569  0.981023  0.957783       NaN   
2012-03-19  73.050003  1908700  ...  1.390202  0.981813  0.954347       NaN   
2012-03-20  72.889999  2086800  ...  1.383802  0.990520  0.960322  0.856604   

                                                                       
                 SAR Williams%R       APO     ROC_5    ROC_20 ROC_100  
Date                                                                   
2011-10-26       NaN        NaN       NaN       NaN       NaN     NaN  
2011-10-27  0.960701        NaN       NaN       NaN       NaN     NaN  
2011-10-28  0.966667        NaN       NaN       NaN       NaN     NaN  
2011-10-31  0.981852        NaN       NaN       NaN       NaN     NaN  
2011-11-01  1.060211        NaN       NaN       NaN       NaN     NaN  
...              ...        ...       ...       ...       ...     ...  
2012-03-14  0.750000  -9.118120  0.018743  0.020148  0.055576     NaN  
2012-03-15  0.736956  -5.324721  0.018389  0.022352  0.083438     NaN  
2012-03-16  0.730087  -0.163233  0.018960  0.032126  0.075568     NaN  
2012-03-19  0.724778  -1.115500  0.019335  0.041256  0.075955     NaN  
2012-03-20  0.726621  -5.640902  0.019721  0.032046  0.076360     NaN  

[100 rows x 380 columns]

At this point, we have a 3D nested dataframe, split by tickers and then by columns. To put this data in CSV files, we will create a CSV file for each ticker containing the relevant information.

In [41]:


for ticker in tickers:
    print(f"Saving data for {ticker}")
    
    tickerdata = data[ticker].dropna()
    tickerdata.to_csv(f'./data/{ticker}.csv') #write to csv file

print("======================")
print("Saved stocks to CSV files. Take a look inside /data for them.")

Saving data for AMZN
Saving data for TSLA
Saving data for BABA
Saving data for HD
Saving data for TM
Saving data for NKE
Saving data for MCD
Saving data for LOW
Saving data for SBUX
Saving data for JD
Saving data for PDD
Saving data for BKNG
Saving data for GM
Saving data for MELI
Saving data for TJX
Saving data for NIO
Saving data for F
Saving data for LULU
Saving data for HMC
Saving data for CMG
Saved stocks to CSV files. Take a look inside /data for them.
